In [173]:
import pandas as pd
import altair as alt

In [174]:
dados_federais_campina = pd.read_csv("data\\dados_federais_obitos_cg.csv", encoding='utf-8', error_bad_lines=False)
dados_painelfederal_campina = pd.read_csv("data\\dados_painel-federal_obitos_cg.csv", encoding='UTF-8', error_bad_lines=False)
dados_municipais_campina = pd.read_csv("data\\dados_municipais_cg.csv", encoding='utf-8', index_col='data')

C:\Users\perei\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


# Processando os dados sobre o Numero de Obitos por Dia (Painel Federal)

In [175]:
# Criando um data com a coluna 'data' e a coluna 'obitosNovos'
pf_obitosPorDia = dados_painelfederal_campina[['obitosNovos', 'data']]


In [176]:
# Criando um DataFrame com os dados para o altair
pf_obitosPorDia = pd.DataFrame({
    'data': pf_obitosPorDia['data'].tolist(),
    'obitos_confirmados': pf_obitosPorDia['obitosNovos'].tolist()
})

# Criando uma nova coluna no DataFrame referente a média movel.
pf_obitosPorDia['media_movel'] = pf_obitosPorDia.obitos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
pf_obitosPorDia.set_index('data', inplace=True)


# Processando os dados sobre o Numero de Obitos por Dia (DadosFederais)

In [177]:
# Criando um data com a coluna 'dataNotificacao' e a coluna 'saidaConfirmadaObitos'
df_obitosPorDiaeCNES = dados_federais_campina[['dataNotificacao', 'saidaConfirmadaObitos']]

In [178]:
df_obitosPorDia = {}

for data in df_obitosPorDiaeCNES['dataNotificacao']:
    if data not in df_obitosPorDia:
        ocorrenciasDatas = df_obitosPorDiaeCNES.loc[df_obitosPorDiaeCNES['dataNotificacao'] == data]
        df_obitosPorDia[data] = ocorrenciasDatas['saidaConfirmadaObitos'].sum()

In [179]:
# Criando um DataFrame com os dados para o altair
df_obitosPorDia = pd.DataFrame({
    'data': df_obitosPorDia.keys(),
    'obitos_confirmados': df_obitosPorDia.values()
})

# Criando uma nova coluna no DataFrame referente a média movel.
df_obitosPorDia['media_movel'] = df_obitosPorDia.obitos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
df_obitosPorDia.set_index('data', inplace=True)

# Processando os sobre o Numero de Obitos por Dia (Dados Municipais)

In [180]:
# Criando um data com a coluna 'data' e a coluna 'obitosNovos'
dm_obitosPorDia = dados_municipais_campina['obitosPorDia']

In [181]:
# Criando um DataFrame com os dados para o altair
dm_obitosPorDia = pd.DataFrame({
    'data': dm_obitosPorDia.index.tolist(),
    'obitos_confirmados': dm_obitosPorDia.values.tolist()
})

# Criando uma nova coluna no DataFrame referente a média movel.
dm_obitosPorDia['media_movel'] = dm_obitosPorDia.obitos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
dm_obitosPorDia.set_index('data', inplace=True)


In [182]:
def exclui_nao_contidos(df_conjunto, df_subconjunto):
    for i in df_conjunto.index:
        if i not in df_subconjunto.index:
            df_conjunto = df_conjunto.drop(i)
    return df_conjunto

In [183]:
pf_obitosPorDia = exclui_nao_contidos(pf_obitosPorDia, dm_obitosPorDia)
df_obitosPorDia = exclui_nao_contidos(df_obitosPorDia, dm_obitosPorDia)

# Comparando os dados Federais e Municipais

In [184]:
obitosPorDia = pd.DataFrame({
  'data': pf_obitosPorDia.index.tolist(),
  'Painel Federal': pf_obitosPorDia['obitos_confirmados'].tolist(),
  'Federal': df_obitosPorDia['obitos_confirmados'].tolist(),
  'Municipal': dm_obitosPorDia['obitos_confirmados'].tolist()
}).melt(id_vars=['data'], value_vars=['Painel Federal', 'Federal', 'Municipal'],
        var_name='fonte_dados', value_name='aplic_por_dia')

In [185]:
alt.Chart(obitosPorDia).mark_line().encode(
    alt.X('data:T', title='Data'),
    alt.Y('aplic_por_dia:Q', title='Nº de Obitos'),
    alt.Color('fonte_dados:N', title='Base dos Dados')
).properties(
    width=700,
    title='Quantidade de Obitos Confirmados por Dia'
)

alt.Chart(...)

In [187]:
obitosPorDiaMedMovel = pd.DataFrame({
  'data': pf_obitosPorDia.index.tolist(),
  'Painel Federal': pf_obitosPorDia['media_movel'].tolist(),
  'Federal': df_obitosPorDia['media_movel'].tolist(),
  'Municipal': dm_obitosPorDia['media_movel'].tolist()
}).melt(id_vars=['data'], value_vars=['Painel Federal', 'Federal', 'Municipal'],
        var_name='fonte_dados', value_name='aplic_por_dia')

In [188]:
alt.Chart(obitosPorDiaMedMovel).mark_line(point=True).encode(
    alt.X('data:T', title='Data'),
    alt.Y('aplic_por_dia:Q', title='Nº de Obitos'),
    alt.Color('fonte_dados:N', title='Base dos Dados')
).properties(
    width=750,
    title='Média Movel de Obitos Confirmados por Dia (Comparação)'
)

alt.Chart(...)

In [189]:
pf_obitosTotais = pf_obitosPorDia['obitos_confirmados'].sum()
df_obitosTotais = df_obitosPorDia['obitos_confirmados'].sum()
dm_obitosTotais = dm_obitosPorDia['obitos_confirmados'].sum()

In [190]:
totalObitos = pd.DataFrame({
    'base': ['Painel Federal', 'Federal', 'Municipal'],
    'total': [pf_obitosTotais, df_obitosTotais, dm_obitosTotais]
})

In [191]:
alt.Chart(totalObitos).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Obitos Confirmados'),
    alt.Color('base:N', title='Base de Dados')
).properties(
    width=150,
    title='Total de Obitos Confirmados (Comparação)'
)

alt.Chart(...)